In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
COVID19_line_list_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv")
COVID19_open_line_list = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")
covid_19_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")
time_series_covid_19_confirmed = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv")
time_series_covid_19_deaths = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv")
time_series_covid_19_recovered = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv")

<div>
    <h3>Dataset: covid_19_data.csv</h3>
    <dl>
        <dt>Number of rows</dt>
        <dd>4718<dd>
        <dt>Number of columns</dt>
        <dd>8<dd>
        <dt>Column names</dt>
        <dd>SNo, ObservationDate, Province/State, Country/Region, Last Update, Confirmed, Deaths, Recovered<dd>
        <dt>Columns with missing data</dt>
        <dd>Province/State</dd>
    </dl>
</div> 


In [ ]:
covid_19_data.head(10)

In [ ]:
covid_19_data.tail(10)

In [ ]:
covid_19_data.info()

<div>
    <h3>Statistics on Confirmed Covid-19 Cases</h3>
    <h4>Global Statistics</h4>
    <p>The total number of confirmed cases is 125865 on 03/11/20.</p>
    <p>The daily percentage increase in the number of confirmed cases was <em>6.14%</em> on 03/11/2020.</p>
    <p>The highest daily percentage increase in the number of confirmed cases was <em>90.57%</em> on 01/28/2020.</p>
    <p>The lowest daily percentage increase in the number of confirmed cases was <em>0.49%</em> on 02/23/2020.</p>
    <p>The average daily percentage increase in the number of confirmed cases was <em>12.86%</em> between 01/22/2020 and 03/11/2020.</p>
    <p>Death Rate from Covid-19: 3.67%</p>
    <p>Recovery Rate from Covid-19: 53.23%</p>
    <h4>Top 10 Countries with Highest Number of Covid-19 Cases</h4>
    <ol>
        <li>Mainland China</li>
        <li>Italy</li>
        <li>South Korea</li>
        <li>France</li>
        <li>Spain</li>
        <li>Germany</li>
        <li>Switzerland</li>
        <li>Japan</li>
        <li>Norway</li>
        <li>Netherland</li>
    </ol>
</div>

In [ ]:
# data grouped by time
data_by_time = covid_19_data[["ObservationDate", "Confirmed", "Deaths", "Recovered"]].groupby(by=["ObservationDate"]).sum().sort_values(by=["Confirmed"])
data_by_time

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6, 6))

print(f"The total number of confirmed cases is {data_by_time['Confirmed'][-1]} on {data_by_time.index[-1]}.")

plt.plot(list(range(len(data_by_time))), data_by_time["Confirmed"])
plt.title("Total Number of Covid-19 Cases")
plt.xlabel("Days since 01/22/2020")
plt.ylabel("Number of Cases")
plt.show()

In [ ]:
from datetime import date, timedelta
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6, 6))

t = list(range(1, len(data_by_time)))
confirmed = data_by_time["Confirmed"].values
growth_percentage = 100 * (confirmed[1:] - confirmed[:-1]) / confirmed[:-1]

print(f"The daily percentage increase in the number of confirmed cases was {np.around(growth_percentage[-1], 2)}% on {data_by_time.index[-1]}.")
print(f"The highest daily percentage increase in the number of confirmed cases was {np.around(np.max(growth_percentage), 2)}% on {(date(2020, 1, 22) + timedelta(days=int(np.argmax(growth_percentage) + 1))).strftime('%m/%d/%Y')}.")
print(f"The lowest daily percentage increase in the number of confirmed cases was {np.around(np.min(growth_percentage), 2)}% on {(date(2020, 1, 22) + timedelta(days=int(np.argmin(growth_percentage) + 1))).strftime('%m/%d/%Y')}.")
print(f"The average daily percentage increase in the number of confirmed cases was {np.around(np.mean(growth_percentage), 2)}% between {data_by_time.index[0]} and {data_by_time.index[-1]}.")

plt.plot(t, growth_percentage)
plt.title("Daily Percentage Increase in the Number of Confirmed Cases")
plt.xlabel("Days since 01/22/2020")
plt.ylabel("Percentage Increase(%)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6, 6))

width = 0.5
recovered = data_by_time["Recovered"].values[-1]
deaths = data_by_time["Deaths"].values[-1]
ongoing = data_by_time["Confirmed"].values[-1] - recovered - deaths

print(f"Death Rate from Covid-19: {np.around(deaths / (recovered + deaths + ongoing) * 100, 2)}%")
print(f"Recovery Rate from Covid-19: {np.around(recovered / (recovered + deaths + ongoing) * 100, 2)}%")

p1 = plt.bar([0], ongoing, width)
p2 = plt.bar([0], recovered, width, bottom=ongoing)
p3 = plt.bar([0], deaths, width, bottom=recovered + ongoing)

plt.title(f"Total Number of Cases as of {data_by_time.index[-1]}")
plt.ylabel("Number of Cases")
plt.xlim((-1, 1))
plt.xticks(())
plt.legend((p1[0], p2[0], p3[0]), ('Ongoing', 'Recovered', 'Death'))
plt.show()

In [ ]:
# data grouped by countries
data_by_country = covid_19_data[["Country/Region", "Confirmed", "Recovered", "Deaths"]].groupby(by=["Country/Region"]).max().sort_values(by=["Confirmed"], ascending=False)
data_by_country['Ongoing'] = data_by_country["Confirmed"] - data_by_country["Recovered"] - data_by_country["Deaths"]
data_by_country.head()

In [ ]:
n = 15
data_by_country[["Ongoing", "Recovered", "Deaths"]][:n].plot.bar(figsize=(10, 20), stacked=True)
plt.show()

<div>
    <h3>Preparation of Data</h3>
</div>

In [ ]:
import pandas as pd
COVID19_line_list_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv")
COVID19_open_line_list = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")
covid_19_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")
time_series_covid_19_confirmed = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv")
time_series_covid_19_deaths = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv")
time_series_covid_19_recovered = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv")

In [ ]:
# data cleansing
titles = ["Confirmed", "Deaths", "Recovered"]
combined = [time_series_covid_19_confirmed, time_series_covid_19_deaths, time_series_covid_19_recovered]

for df in combined:
    df.fillna(value="Unknown", inplace=True)
    df.sort_values(by=["Country/Region", "Province/State"], inplace=True)
    df.reset_index(drop=True, inplace=True)

combined[0].head(10)

In [ ]:
location_names = time_series_covid_19_confirmed["Country/Region"] + " " + time_series_covid_19_confirmed["Province/State"]
time_series_data = []

for title, df in zip(titles, combined):
    data = df.loc[:, "1/22/20":].set_index(location_names).transpose()
    data.columns = [col + " " + title for col in data.columns]
    time_series_data.append(data)
    
time_series_data[0].head(10)

<div>
    <h3>Modelling the Data</h3>
</div>

In [ ]:
n_locations = time_series_data[0].shape[1]
print(f"Number of locations in dataset: {n_locations} ")

In [ ]:
from sklearn.metrics import mean_absolute_error

print("Using value at time = t - 1 to predict value at time = t\n")

for title, df in zip(titles, time_series_data):
    y = df.values[1:]
    y_hat = df.shift(1).values[1:]
    
    print(f"{title} cases")
    print(f"Mean absolute error over all days: {np.around(mean_absolute_error(y, y_hat), 4)}")
    print(f"Mean absolute error over past 20 days: {np.around(mean_absolute_error(y[-20:], y_hat[-20:]), 4)}")
    print(f"Mean absolute error over past 10 days: {np.around(mean_absolute_error(y[-10:], y_hat[-10:]), 4)}")
    print(f"Mean absolute error over past 5 days: {np.around(mean_absolute_error(y[-5:], y_hat[-5:]), 4)}\n")

In [ ]:
def to_supervised_learning_data(dfs, n_lags, n_leads, n_vars):
    data = []
    
    for lag in range(n_lags, 0, -1):
        for df in dfs:
            x = df.shift(lag)
            x.columns = [f"{col} at time=t-{lag}" for col in df.columns]
            data.append(x)
            
    for df in dfs:
        y = df.copy()
        y.columns = [f"{col} at time=t" for col in df.columns]
        data.append(y)

    for lead in range(1, n_leads):
        for df in dfs:
            y = df.shift(-lead)
            y.columns = [f"{col} at time=t+{lead}" for col in df.columns]
            data.append(y)

    data = pd.concat(data, axis=1).dropna()
    return data.iloc[:, : n_lags * n_vars * 3], data.iloc[:, n_lags * n_vars * 3:]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

n_leads = 1
n_lags = 2
n_hidden = 1024
n_df = len(time_series_data)

X, Y = to_supervised_learning_data(time_series_data, n_lags, n_leads, n_locations)
x_train = X.values.reshape((len(X), n_lags, n_df * n_locations))
y_train = Y.values             

n_epochs = 500000
initial_lr = 1e-2
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(n_lags, n_df * n_locations)))
model.add(Dense(n_leads * n_df * n_locations))
optimizer = optimizers.Adam(lr=initial_lr)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-6)
model.compile(optimizer=optimizer, loss='mae')
model.fit(x_train, y_train, epochs=n_epochs, verbose=1)

In [ ]:
model.save("lstm_model.h5")